# Plots for Thesis

### Setup Notebook

Before running this interactive notebook, check the following parameters:
- `ENV`: Select your execution environment `"LOCAL"` (VS Code, Jupyter Notebook) or `"COLAB"` (Google Colab).
- `PLT_INTERACTIVE`: Select whether the plots should be interactive to allow zooming, panning and resizing. In general, this can stay activated.

In [ ]:
#@title { display-mode: "form" }

ENV = "LOCAL" #@param ["LOCAL", "COLAB"]
PLT_INTERACTIVE = True #@param {type:"boolean"}

# setup environment
LOCAL = "LOCAL"
COLAB = "COLAB"
if ENV == COLAB:
    %cd /content
    !git clone https://github.com/danielyxyang/active_reconstruction.git
    %cd /content/active_reconstruction
    %pip install -q -r requirements.txt
    %cd /content/active_reconstruction/src
elif ENV == LOCAL:
    %cd ../src

# setup interactive plots
if PLT_INTERACTIVE:
    if ENV == COLAB:
        from google.colab import output
        output.enable_custom_widget_manager()
    %matplotlib widget
else:
    %matplotlib inline

# ensure automatic reload of imported modules
%load_ext autoreload
%autoreload 2


### CUSTOM SETUP ###

import math
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import FancyArrowPatch, Arc
from sklearn.gaussian_process.kernels import DotProduct

import parameters as params
import algorithms.gp as gp
import algorithms.objectives as objectives
from algorithms.algorithms import TRUE_ALGORITHM
import simulation.objects as objects
from simulation.camera import Camera
from simulation.simulation import Simulation
from simulation.plotter import SimulationPlotter
from utils.math import polar_to_cartesian, is_in_range
from utils.plotting import PlotSaver, GIFSaver, MultipleTicks

plt.ioff() # prevent figures to be displayed without calling plt.show() or display()
SimulationPlotter.set_interactive(PLT_INTERACTIVE)
PlotSaver.set_interactive(PLT_INTERACTIVE)

# setup PlotSaver
PlotSaver.setup(
    output=os.path.join(".", "..", "output"),
    textwidth=5.10585305,
    fontsize=9,
    latex_preamble="\n".join([
        r"\usepackage[utf8]{inputenc}",
        r"\usepackage[T1]{fontenc}",
        r"\usepackage{lmodern}",      # for 8-bit Latin Modern font
        r"\usepackage[sc]{mathpazo}", # for Palatino font
        r"\usepackage{amsmath,amssymb,amsfonts,mathrsfs}",
    ]),
    params={
        "lines.linewidth": 1, # default: 1.5
        "axes.labelpad": 1, # default: 4
    }
)
PlotSaver.enable_latex()
# PlotSaver.enable_save_all("pdf")

# default object and camera
OBJECT = lambda: objects.PolygonObject.build("convex")
CAMERA = lambda: Camera(1.6)
KERNEL = lambda: gp.build_kernel_matern_periodic(sigma=1.5, l=0.2, nu=1.5)

YLIM_1 = (0, 1.1)

# plotting utils

def print_lim(i):
    fig = plt.figure(i)
    print("({:.2f}, {:.2f})".format(*fig.gca().get_xlim()))
    print("({:.2f}, {:.2f})".format(*fig.gca().get_ylim()))

def transfer_lim(i, j):
    fig_from = plt.figure(i)
    fig_to = plt.figure(j)
    fig_to.gca().set_xlim(fig_from.gca().get_xlim())
    fig_to.gca().set_ylim(fig_from.gca().get_ylim())
    fig_to.canvas.draw()
    fig_to.canvas.flush_events()

def pi_ticks(denominator=1):
    ticks = MultipleTicks(
        denominator=denominator, number=np.pi, latex="\pi",
        number_in_frac=False, fracformat=r"\textstyle\frac{%s}{%s}",
    )
    return dict(locator=ticks.locator(), formatter=ticks.formatter())

def arrowprops(rad="0.2"):
    return dict(arrowstyle="->", connectionstyle="arc3,rad={}".format(rad), shrinkA=1, linewidth=0.5)

def build_arrowpatch_length(posA, posB, linewidth=1, width=None, **kwargs):
    if width is None:
        width = 3 * linewidth
    return FancyArrowPatch(posA=posA, posB=posB, arrowstyle="|-|,widthA={},widthB={}".format(width, width), shrinkA=0, shrinkB=0, linewidth=linewidth, zorder=2, **kwargs)

## Plots for Chapter 2

### Fig. 2.1: background-gp-kernels-stationarity

In [ ]:
np.random.seed(0)

def plot():
    plots = []
    
    def make_plot(k, name):
        # evaluate GP
        x = np.linspace(-10, 10, 500)
        gproc = gp.GaussianProcess(m, k, x_eval=x)
        lower, upper = gproc.confidence_boundary()
        f = gproc.sample(n=3)
        # plot GP and samples
        fig, axis = PlotSaver.create(
            xmargin=0,
            ylim=(-10, 10),
        )
        axis.plot(x, f.T)
        axis.plot(x, gproc.mean, color="gray", linestyle="--")
        axis.fill_between(x, lower, upper, color="gray", alpha=0.2)
        plots.append((fig, "background-gp-kernels-stationarity-{}".format(name)))

    m = lambda x: 0
    k1_rbf = gp.build_kernel_rbf(sigma=1, l=1)
    k1_dot = DotProduct(sigma_0=1)
    
    make_plot(k1_rbf, "stationary")
    make_plot(lambda X1, X2: k1_dot(X1, X2) * k1_rbf(X1,X2), "nonstationary")

    PlotSaver.finish(
        plots,
        relsize=0.45,
        save=False,
    )
plot()

### Fig. 2.2: background-gp-kernels-parameters

In [ ]:
np.random.seed(1)

def plot():
    plots = []
    
    def make_plot(k, name):
        # plot kernel
        r = np.linspace(-10, 10, 500)[:, np.newaxis]
        fig, axis = PlotSaver.create(
            xmargin=0,
            ylim=YLIM_1,
            yticks=dict(ticks=[0, 1], labels=["0", r"$\sigma_f^2$"]),
        )
        axis.plot(r, gp.eval_kernel(k, r))
        plots.append((fig, "background-gp-kernels-parameters-{}-kernel".format(name)))
        # evalute GP
        x = np.linspace(-10, 10, 500)
        gproc = gp.GaussianProcess(m, k, x_eval=x)
        lower, upper = gproc.confidence_boundary()
        f = gproc.sample(n=3)
        # plot GP and samples
        fig, axis = PlotSaver.create(
            xmargin=0,
            ylim=(-3, 3),
            yticks=dict(ticks=[-2, 2], labels=[r"$-2\sigma_f$", r"$2\sigma_f$"]),
        )
        axis.plot(x, f.T)
        axis.plot(x, gproc.mean, color="gray", linestyle="--")
        axis.fill_between(x, lower, upper, color="gray", alpha=0.2)
        plots.append((fig, "background-gp-kernels-parameters-{}-samples".format(name)))

    m = lambda x: 0
    k_rbf1 = gp.build_kernel_rbf(sigma=1, l=1)
    k_rbf2 = gp.build_kernel_rbf(sigma=1, l=2)

    make_plot(k_rbf1, "lengthshort")
    y = gp.eval_kernel(k_rbf1, [[1]])[0]
    plots[-2][0].gca().add_patch(build_arrowpatch_length((0, y), (1, y), linewidth=0.75))
    plots[-2][0].gca().annotate(r"$l$", (0.5, 0.47))
    x, y = -0.6, 2.2
    plots[-1][0].gca().add_patch(build_arrowpatch_length((x, y), (x+1, y), linewidth=0.75))
    plots[-1][0].gca().annotate(r"$l$", (x+0.4, y+0.2))
    
    make_plot(k_rbf2, "lengthlong")
    y = gp.eval_kernel(k_rbf2, [[2]])[0]
    plots[-2][0].gca().add_patch(build_arrowpatch_length((0, y), (2, y), linewidth=0.75))
    plots[-2][0].gca().annotate(r"$l$", (1, 0.47))
    x, y = -4.0, 0.7
    plots[-1][0].gca().add_patch(build_arrowpatch_length((x, y), (x+2, y), linewidth=0.75))
    plots[-1][0].gca().annotate(r"$l$", (x+0.8, y+0.2))

    PlotSaver.finish(
        plots[0::2] + plots[1::2],
        relsize=0.45,
        ncols=2,
        save=False,
    )
plot()

### Fig. 2.3: background-gp-kernels-rbf-matern

In [ ]:
np.random.seed(0)

def plot():
    plots = []
    
    def make_plot(k, name):
        # plot kernel
        r = np.linspace(-5.5, 5.5, 1000)[:, np.newaxis]
        fig, axis = PlotSaver.create(
            xmargin=0,
            ylim=YLIM_1,
            yticks=dict(ticks=[0, 1], labels=["0", r"$\sigma_f^2$"]),
        )
        axis.plot(r, gp.eval_kernel(k, r))
        plots.append((fig, "background-gp-kernels-{}-1".format(name)))
        # evaluate GP
        x = np.linspace(-5.5, 5.5, 500)
        gproc = gp.GaussianProcess(m, k, x_eval=x)
        lower, upper = gproc.confidence_boundary()
        f = gproc.sample(n=3)
        # plot GP and samples
        fig, axis = PlotSaver.create(
            xmargin=0,
            ylim=(-3, 3),
            yticks=dict(ticks=[-2, 2], labels=[r"$-2\sigma_f$", r"$2\sigma_f$"])
        )
        axis.plot(x, f.T)
        axis.plot(x, gproc.mean, color="gray", linestyle="--")
        axis.fill_between(x, lower, upper, color="gray", alpha=0.2)
        plots.append((fig, "background-gp-kernels-{}-2".format(name)))

    m = lambda x: 0
    k_rbf = gp.build_kernel_rbf(sigma=1)
    k_mat = gp.build_kernel_matern(sigma=1, nu=1.5)

    make_plot(k_rbf, "rbf")
    make_plot(k_mat, "matern")

    PlotSaver.finish(
        plots[0::2] + plots[1::2],
        relsize=0.45,
        ncols=2,
        save=False,
    )
plot()

### Fig. 2.4: background-gp-gpr

In [ ]:
np.random.seed(0)

def plot():
    plots = []
    
    def make_plot(gproc, name):
        lower, upper = gproc.confidence_boundary()
        f = gproc.sample(n=3)
        
        fig, axis = PlotSaver.create(
            xmargin=0,
            ylim=(-3, 3),
            yticks=dict(ticks=[-2, 2], labels=[r"$-2\sigma_f$", r"$2\sigma_f$"]),
        )
        axis.plot(x_eval, f.T)
        axis.plot(x_eval, gproc.mean, color="gray", linestyle="--")
        axis.fill_between(x_eval, lower, upper, color="gray", alpha=0.2)
        axis.plot(gproc.x, gproc.y, color="red", linestyle="none", marker="o", markersize=4)
        plots.append((fig, "background-gp-gpr-{}".format(name)))
    
    # define measurement points
    x, y = np.array([
        [-4, 1.5],
        [1, -1.5],
        [4, 1],
    ]).T
    
    # define GP
    m = lambda x: 0
    k_rbf = gp.build_kernel_rbf(sigma=1)
    x_eval = np.linspace(-5.5, 5.5, 500)
    gproc = gp.GaussianProcess(m, k_rbf, x_eval=x_eval)

    make_plot(gproc, "prior")
    
    np.random.seed(0)
    gproc.update(x, y)
    make_plot(gproc, "posterior-noisefree")
    
    np.random.seed(0)
    gproc.reset()
    gproc.update(x, y, noise=0.5)
    make_plot(gproc, "posterior-noisy")

    PlotSaver.finish(
        plots,
        relsize=0.33,
        save=False,
    )
plot()

## Plots for Chapter 4

### Fig. 4.2: problem-utility

In [ ]:
def plot():
    # invent some marginal utilities
    rounds = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11,])
    n_marginal = np.array([0, 19, 30, 26, 20, 12, 10, 12, 8, 28, 11, 12])
    n_marginal_opt = np.array([0, 43, 41, 40, 37, 33, 30, 29, 25, 20, 10, 4])
    regret_ind = np.array([0, 49, 20, 13, 27, 19, 23, 21, 0, 2, 1, 0])
    
    # compute utility
    n_total = np.cumsum(n_marginal)
    n_total_opt = np.cumsum(n_marginal_opt)
    n_total_gre = n_total + regret_ind

    def annotate_bar(axis, x, y1, y2, text, textoffset=0, side="right", color=None):
        if side == "right":
            ha = "left"
            sign = 1
        elif side == "left":
            ha = "right"
            sign = -1
        axis.add_patch(build_arrowpatch_length(
            (x+sign*0.15, max(y1+2, 2)),
            (x+sign*0.15, max(y2-2, 2)),
            linewidth=0.75,
            color=color,
        ))
        axis.annotate(
            text,
            (x+sign*0.22, (y1+y2)/2 - 4 + textoffset),
            color=color,
            horizontalalignment=ha,
        )

    fig, axis = PlotSaver.create(
        xlabel=r"time $t$", ylabel=r"utility $F(\Theta)$",
        xlim=(0, 7),
        ylim=(0, 215),
    )
    # specify which round to annotate
    end = 4
    # manipulate greedy utility array
    rounds_interl = np.empty(3*rounds.size)
    rounds_interl[0::3] = rounds-1
    rounds_interl[1::3] = rounds
    rounds_interl[2::3] = rounds
    n_total_gre_interl = np.empty(3*rounds.size)
    n_total_gre_interl[0::3] = np.concatenate([[0], n_total[:-1]])
    n_total_gre_interl[1::3] = n_total_gre
    n_total_gre_interl[2::3] = np.nan
    # plot utilities (beyond relevant part)
    axis.plot(rounds_interl[end*3:], n_total_gre_interl[end*3:], "o--", color="gray", alpha=0.5, markersize=4, linewidth=1)
    axis.plot(rounds[end:], n_total_opt[end:], "o:", color="gray", alpha=0.5, markersize=4, linewidth=1)
    axis.plot(rounds[end:], n_total[end:], "o-", color="gray", alpha=0.5, markersize=4, linewidth=1)
    # plot utilities (relevant part)
    axis.plot(rounds_interl[:(end+1)*3], n_total_gre_interl[:(end+1)*3], "o--", color="darkred", markersize=4, linewidth=1, label=r"$F(\theta^*_t\mid\theta_{1:t-1})+F(\theta_{1:t-1})$")
    axis.plot(rounds[:(end+1)], n_total_opt[:(end+1)], "o:", color="red", markersize=4, linewidth=1, label=r"$\left(1-\frac{1}{e}\right)F(\Theta^\star_t)$")
    axis.plot(rounds[:(end+1)], n_total[:(end+1)], "o-", color="blue", markersize=4, linewidth=1, label=r"$F(\theta_{1:t})$")
    # annotate marginal utilities
    for i in range(1, end+1):
        if i == 1:   annotation = r"$F(\theta_{%s})$" % (rounds[i])
        elif i == 2: annotation = r"$F(\theta_{%s}\mid\theta_{%s})$" % (rounds[i], rounds[i]-1)
        else:        annotation = r"$F(\theta_{%s}\mid\theta_{1:%s})$" % (rounds[i], rounds[i-1])
        annotate_bar(axis, rounds[max(end-1,i)], n_total[i-1], n_total[i], annotation, color="tab:blue")
    # annotate previous utility and individual regret regret
    annotate_bar(axis, rounds[end], n_total[0], n_total[end-1], r"$F(\theta_{1:%s})$" % str(end-1), color="blue")
    annotate_bar(axis, rounds[end], n_total[end], n_total_gre[end], r"$r_{ind}(%s)$" % end, textoffset=17, color="darkred")
    # annotate current utility and regret
    annotate_bar(axis, rounds[end] + 1.15, n_total[0], n_total[end], r"$F(\theta_{1:%s})$" % end, color="blue")
    annotate_bar(axis, rounds[end] + 1.15, n_total[end], n_total_opt[end], r"$R(%s)$" % end, textoffset=12, color="red")
    
    # annotate monotonicity
    axis.annotate("monotonic", (2, 42.5), xytext=(1.7, 15), arrowprops=arrowprops())

    PlotSaver.finish(
        (fig, "problem-utility"),
        relsize=1, legend=dict(order=[2,1,0]),
        save=False,
    )

plot()

### Fig. 4.3: problem-worlds

In [ ]:
def plot():
    plots = []

    cam = CAMERA()
    obj = OBJECT()

    plotter = SimulationPlotter(mode="real")
    PlotSaver.set_style(
        plotter.axis,
        title="", xlabel=r"$x$ coordinate [m]", ylabel=r"$y$ coordinate [m]",
        xticks=[-10,0,10],
        yticks=[-10,0,10],
    )
    plotter.plot_camera(cam, show_los=False, show_view_circle=False)
    plotter.plot_object(obj, show_bounds=False)
    plots.append((plotter.fig, "problem-worlds-real"))

    plotter = SimulationPlotter(mode="polar")
    PlotSaver.set_style(
        plotter.axis,
        title="", xlabel=r"polar angle $\varphi$ [rad]", ylabel=r"radial distance $r$ [m]",
        xticks=pi_ticks(),
        yticks=[0,5,10],
    )
    plotter.plot_camera(cam, show_los=False, show_view_circle=False)
    plotter.plot_object(obj, show_bounds=False)
    plots.append((plotter.fig, "problem-worlds-polar"))
    
    PlotSaver.finish(
        plots,
        relsize=0.45, ratio=1,
        save=False,
    )
plot()

### Fig. 4.4: problem-object-camera

In [ ]:
def plot():
    plots = []

    cam = CAMERA()
    obj = OBJECT()

    # a) object and camera in real and polar world
    plotter = SimulationPlotter(mode="real", undecorated=True)
    PlotSaver.set_style(
        plotter.axis,
        xlabel=r"$x$ coordinate",
        ylabel=r"$y$ coordinate",
    )
    plotter.plot_camera(cam)
    plotter.plot_object(obj)
    plotter.axis.annotate(r"$d_{DOF}$", polar_to_cartesian(cam.theta, 0.35*params.CAM_D), xytext=(1.5, -4), arrowprops=arrowprops(rad="0.2"))
    plotter.axis.add_patch(Arc(
        polar_to_cartesian(cam.theta, params.CAM_D), 7, 7,
        theta1=math.degrees(cam.theta+np.pi - params.CAM_FOV_RAD()/2),
        theta2=math.degrees(cam.theta+np.pi + params.CAM_FOV_RAD()/2),
        color="blue", alpha=0.5, linestyle="-", linewidth=1,
    ))
    plotter.axis.annotate(r"$\alpha_{FOV}$", polar_to_cartesian(*cam.camera_to_polar(0.1 * -params.CAM_FOV_RAD()/2, 3.5)), xytext=(-9.5, 8.8), arrowprops=arrowprops(rad="-0.2"))
    plots.append((plotter.fig, "problem-object-camera-a-real"))

    plotter = SimulationPlotter(mode="polar", undecorated=True)
    PlotSaver.set_style(
        plotter.axis,
        xticks=pi_ticks(),
        yticks=[],
        ylabel=r"radial distance $r$",
    )
    plotter.plot_camera(cam)
    plotter.plot_object(obj)
    plotter.axis.annotate(r"$f(\varphi)$", (3.7, 5.2))
    plotter.axis.annotate(r"$\theta$", (cam.theta-0.15, -1.2), annotation_clip=False)
    plotter.axis.annotate(r"$d_{min}$", (5.1, params.OBJ_D_MIN - 1))
    plotter.axis.annotate(r"$d_{max}$", (5.1, params.OBJ_D_MAX - 1))
    plotter.axis.annotate(r"$d_{cam}$", (5.1, params.CAM_D - 1))
    plots.append((plotter.fig, "problem-object-camera-a-polar"))

    # b) discretization and observation model
    with params.local():
        params.GRID_H = 0.5
        obj._discretize()

        plotter = SimulationPlotter(mode="real", undecorated=True)
        PlotSaver.set_style(
            plotter.axis,
            xlabel=r"$x$ coordinate",
            ylabel=r"$y$ coordinate",
            xlim=[-2.5, 3.5],
            ylim=[4.5, 10.5],
        )
        plotter.plot_grid()
        plotter.plot_object(obj, show_pixels=True, show_points=True, n=1000)
        plotter.plot_camera(cam)
        plotter.plot_points(cam.compute_observation(obj.surface_points), color="red")
        plotter.axis.add_patch(build_arrowpatch_length((1.75, 8.75), (1.75+params.GRID_H, 8.75), linewidth=0.5))
        plotter.axis.annotate(r"$h$", (1.9, 9))
        some_pixel = polar_to_cartesian(*obj.surface_points[:, 18])
        plotter.axis.annotate(r"$x_i$", some_pixel + [0.05,-0.15], xytext=(2.5, 5), arrowprops=arrowprops(rad="0.2"))
        center_obs = (polar_to_cartesian(*obj.surface_points[:, 24]) + polar_to_cartesian(*obj.surface_points[:, 25])) / 2
        plotter.axis.annotate(r"$o(\theta)$", center_obs + [0.1,-0.3], xytext=(0.3, 5), arrowprops=dict(arrowstyle="-[,widthB=2.65", connectionstyle="arc3,rad=-0.64", linewidth=0.5))
        plots.append((plotter.fig, "problem-object-camera-b"))
    
    PlotSaver.finish(
        plots,
        relsize=0.33, ratio=1,
        save=False,
    )
plot()

### Fig. 4.5: problem-different-fovs

In [ ]:
def plot():
    plots = []

    with params.local():
        for i, reldof in enumerate([0.8, 1, 1.2]):
            params.CAM_DOF = reldof * params.CAM_D
            # setup setting
            cam = CAMERA()
            obj = OBJECT()
            rays = []
            for ray_alpha in [1/3, 2/3]:
                rays.append(cam.camera_to_polar(ray_alpha * params.CAM_FOV_RAD()/2, np.linspace(0, params.CAM_DOF, 100)))
                rays.append(cam.camera_to_polar(-ray_alpha * params.CAM_FOV_RAD()/2, np.linspace(0, params.CAM_DOF, 100)))
            ray_style = dict(color="blue", alpha=0.2, linestyle="--", linewidth=1)
            # plot FOV in real world
            plotter = SimulationPlotter(mode="real", undecorated=True)
            PlotSaver.set_style(
                plotter.axis,
                xlabel=r"$x$ coordinate",
                ylabel=r"$y$ coordinate",
            )
            plotter.plot_camera(cam)
            plotter.plot_object(obj, show_bounds=False)
            for ray in rays:
                plotter.axis.plot(*polar_to_cartesian(*ray), **ray_style)
            plots.append((plotter.fig, "problem-different-fovs-{}-real".format(i+1)))
            # plot FOV in polar world
            plotter = SimulationPlotter(mode="polar", undecorated=True)
            PlotSaver.set_style(
                plotter.axis,
                xticks=pi_ticks(),
                yticks=[],
                ylabel=r"radial distance $r$",
            )
            plotter.plot_camera(cam)
            plotter.plot_object(obj, show_bounds=False)
            for ray in rays:
                plotter.axis.plot(ray[0], ray[1], **ray_style)
                plotter.axis.plot(ray[0] + 2*np.pi, ray[1], **ray_style)
                plotter.axis.plot(ray[0] - 2*np.pi, ray[1], **ray_style)
            plots.append((plotter.fig, "problem-different-fovs-{}-polar".format(i+1)))
        
        PlotSaver.finish(
            plots[0::2] + plots[1::2],
            relsize=0.33, ratio=1,
            ncols=3,
            save=False,
        )
plot()

### Fig. 4.6: problem-perspective-alg

In [ ]:
def plot():
    plots = []

    def make_plot(mode, t):
        plotter = SimulationPlotter(mode=mode, undecorated=True)
        PlotSaver.set_style(
            plotter.axis,
            xticks=[] if mode == "real" else pi_ticks(0.5),
            xlabel=r"$x$ coordinate" if mode == "real" else "",
            ylabel=r"$y$ coordinate" if mode == "real" else r"radial distance $r$",
        )
        plotter.plot_object(simulation.obj, show_object=False)   
        plotter.plot_camera(simulation.camera)    
        plotter.plot_confidence(simulation.algorithm.gp)
        plotter.plot_points(simulation.algorithm.observations.observed_points, color="green", markersize=1, name="previous")
        plotter.plot_points(simulation.camera.compute_observation(simulation.obj.surface_points), color="red", markersize=1, name="current")
        if mode == "polar":
            plotter.axis.annotate(r"$\theta_{}$".format(t), (simulation.camera.theta-0.15, -1.2), annotation_clip=False)
        plots.append((plotter.fig, "problem-perspective-alg-t{}-{}".format(t, mode)))

    # setup simulation
    cam = Camera()
    obj = objects.PolygonObject.build("star")
    k = gp.build_kernel_matern_periodic(sigma=1.5, l=0.2, nu=1.5)
    simulation = Simulation.build(obj, cam, k, TRUE_ALGORITHM)

    # run simulation and make plots
    thetas = [4.38, 3.17, 1.84]
    for t, theta in enumerate(thetas, start=1):
        simulation.move_camera(theta)
        make_plot("real", t)
        make_plot("polar", t)
        simulation.take_measurement()

    # annotate plots
    plots[1][0].gca().annotate(r"$l_t(\varphi)$", (0.9, 0.9))
    plots[1][0].gca().annotate(r"$\mu_t(\varphi)$", (0.9, 5.5))
    plots[1][0].gca().annotate(r"$u_t(\varphi)$", (0.9, 8.5))

    PlotSaver.finish(
        plots[0::2] + plots[1::2],
        relsize=0.33, ratio=1,
        ncols=3,
        save=False,
    )


with params.local():
    params.GRID_H = 0.5
    plot()

### Fig. 4.7: problem-true-objective

In [ ]:
def plot():
    plots = []
    
    def make_plot(name, algorithm_name, mode="real"):
        # setup simulation
        cam = Camera()
        obj = objects.PolygonObject.build("star")
        k = gp.build_kernel_matern_periodic(sigma=1.5, l=0.2, nu=1.5)
        if algorithm_name == "ObservedSurface":
            simulation = Simulation.build(obj, cam, k)
        else:
            simulation = Simulation.build(obj, cam, k, algorithm_name)
        # run simulation
        thetas = [4.38, 3.17, 1.84]
        simulation.run(thetas[:-1])
        simulation.move_camera(thetas[-1])

        # plot general components
        plotter = SimulationPlotter(mode=mode, undecorated=True)
        PlotSaver.set_style(
            plotter.axis,
            xlabel=r"$x$ coordinate",
            ylabel=r"$y$ coordinate",
        )
        plotter.plot_object(simulation.obj, show_object=False, show_bounds=False)   
        for theta in thetas[:-1]:
            plotter.plot_camera(Camera(theta), show_los=False, show_view_circle=False, color="skyblue", name=theta)
        plotter.plot_camera(simulation.camera)    
        plotter.plot_confidence(simulation.algorithm.gp)
        # plot utility
        if algorithm_name == "ObservedSurface":
            plotter.axis.annotate(r"$\theta_1$", (-5.43, -10.2))
            plotter.axis.annotate(r"$\theta_2$", (-9.29, -1.9))
            plotter.axis.annotate(r"$\theta_3$", (-4.4, 7.6))

            plotter.plot_points(simulation.algorithm.observations.observed_points, color="red", markersize=1, name="previous")

            estimates = simulation.camera.compute_observation(simulation.obj.surface_points)
            plotter.plot_points(estimates, color="red", markersize=1, name="estimate")
        # plot true utility
        elif algorithm_name == TRUE_ALGORITHM:
            plotter.plot_points(simulation.algorithm.observations.observed_points, color="green", markersize=1, name="previous")

            estimates = simulation.algorithm_opt.compute_estimate_points(simulation.camera)
            plotter.plot_points(estimates, color="red", markersize=1, name="estimate")
        
        plots.append((plotter.fig, "problem-true-objective-{}".format(name)))

    make_plot("full", "ObservedSurface")
    make_plot("marginal", TRUE_ALGORITHM)
    PlotSaver.finish(
        plots,
        relsize=0.45, ratio=1,
        save=False,
    )

with params.local():
    params.GRID_H = 0.5
    plot()

## Plots for Chapter 5

### Fig. 5.2: design-gp-nonperiodic-periodic

In [ ]:
def plot():
    plots = []
    
    def make_plot(k1, k2, name):
        fig, axis = PlotSaver.create(
            xmargin=0,
            ylim=YLIM_1,
            xticks=pi_ticks(0.5),
            yticks=dict(ticks=[0, 1], labels=["0", r"$\sigma_f^2$"]),
        )
        axis.plot(r, gp.eval_kernel(k1, r))
        plots.append((fig, "design-gp-{}1".format(name)))

        simulation = Simulation.build(obj, cam, k2)
        simulation.take_measurement()
        
        plotter = SimulationPlotter(mode="polar", undecorated=True)
        PlotSaver.set_style(
            plotter.axis,
            xticks=pi_ticks(),
            ylabel=r"radial distance $r$",
        )
        plotter.plot_object(simulation.obj, show_bounds=False)
        plotter.plot_camera(simulation.camera)
        plotter.plot_confidence(simulation.algorithm.gp)
        plotter.plot_points(simulation.algorithm.observations.observed_points, color="green", markersize=2, name="observations")
        lower, upper = simulation.algorithm.gp.confidence_boundary()
        plotter.axis.plot(simulation.algorithm.gp.x_eval, lower, color="dimgray")
        plotter.axis.plot(simulation.algorithm.gp.x_eval, upper, color="dimgray")
        plots.append((plotter.fig, "design-gp-{}2".format(name)))

        plotter = SimulationPlotter(mode="real", undecorated=True)
        PlotSaver.set_style(
            plotter.axis,
            xlim=(0.25, 8.65),
            ylim=(-2.65, 5.75),
            xlabel=r"$x$ coordinate",
            ylabel=r"$y$ coordinate",
        )
        plotter.plot_object(simulation.obj, show_bounds=False)
        plotter.plot_camera(simulation.camera)
        plotter.plot_confidence(simulation.algorithm.gp)
        plotter.plot_points(simulation.algorithm.observations.observed_points, color="green", markersize=2, name="observations")
        lower, upper = simulation.algorithm.gp.confidence_boundary()
        plotter.axis.plot(*polar_to_cartesian(simulation.algorithm.gp.x_eval, lower), color="dimgray")
        plotter.axis.plot(*polar_to_cartesian(simulation.algorithm.gp.x_eval, upper), color="dimgray")
        plots.append((plotter.fig, "design-gp-{}3".format(name)))

    obj = OBJECT()
    cam = Camera(0.58)
    
    r = np.linspace(-2*np.pi, 2*np.pi, 500)[:, np.newaxis]
    k1_nonperiodic = gp.build_kernel_matern(nu=1.5)
    k1_periodic = gp.build_kernel_matern_periodic(nu=1.5)
    k2_nonperiodic = gp.build_kernel_matern(nu=1.5, sigma=1.5, l=0.2)
    k2_periodic = gp.build_kernel_matern_periodic(nu=1.5, sigma=1.5, l=0.2)

    make_plot(k1_nonperiodic, k2_nonperiodic, name="nonperiodic")
    make_plot(k1_periodic, k2_periodic, name="periodic")

    PlotSaver.finish(
        plots,
        relsize=0.33, ratio=1,
        ncols=3,
        save=False,
    )

with params.local():
    params.GRID_H = 0.3
    plot()

### Fig. 5.3: design-gp-rbf-matern

In [ ]:
def plot():
    plots = []

    obj = OBJECT()
    cam = Camera(1.07)

    r = np.linspace(-2*np.pi, 2*np.pi, 500)[:, np.newaxis]
    k1_rbf = gp.build_kernel_rbf()
    k2_rbf = gp.build_kernel_rbf(sigma=1.5, l=0.25)
    k1_mat12 = gp.build_kernel_matern(nu=0.5)
    k2_mat12 = gp.build_kernel_matern(nu=0.5, sigma=1.5, l=0.25)
    
    def make_plot(k1, k2, name):
        fig, axis = PlotSaver.create(
            xmargin=0,
            ylim=YLIM_1,
            xticks=pi_ticks(0.5),
            yticks=dict(ticks=[0, 1], labels=["0", r"$\sigma_f^2$"]),
        )
        axis.plot(r, gp.eval_kernel(k1, r))
        plots.append((fig, "design-gp-{}1".format(name)))

        simulation = Simulation.build(obj, cam, k2)
        simulation.take_measurement()

        plotter = SimulationPlotter(mode="polar", undecorated=True)
        PlotSaver.set_style(
            plotter.axis,
            xlim=(0.52, 1.60),
            ylim=(3.99, 10.71),
            xticks=pi_ticks(4),
            ylabel=r"radial distance $r$",
        )
        plotter.plot_object(simulation.obj, show_bounds=False)
        plotter.plot_camera(simulation.camera)
        plotter.plot_confidence(simulation.algorithm.gp)
        plotter.plot_points(simulation.algorithm.observations.observed_points, color="green", markersize=2, name="observations")
        lower, upper = simulation.algorithm.gp.confidence_boundary()
        plotter.axis.plot(simulation.algorithm.gp.x_eval, lower, color="dimgray")
        plotter.axis.plot(simulation.algorithm.gp.x_eval, upper, color="dimgray")
        plots.append((plotter.fig, "design-gp-{}2".format(name)))

        plotter = SimulationPlotter(mode="real", undecorated=True)
        PlotSaver.set_style(
            plotter.axis,
            xlim=(1.06, 5.43),
            ylim=(3.45, 7.81),
            xlabel=r"$x$ coordinate",
            ylabel=r"$y$ coordinate",
        )
        plotter.plot_object(simulation.obj, show_bounds=False)
        plotter.plot_camera(simulation.camera)
        plotter.plot_confidence(simulation.algorithm.gp)
        plotter.plot_points(simulation.algorithm.observations.observed_points, color="green", markersize=2, name="observations")
        lower, upper = simulation.algorithm.gp.confidence_boundary()
        plotter.axis.plot(*polar_to_cartesian(simulation.algorithm.gp.x_eval, lower), color="dimgray")
        plotter.axis.plot(*polar_to_cartesian(simulation.algorithm.gp.x_eval, upper), color="dimgray")
        plots.append((plotter.fig, "design-gp-{}3".format(name)))

    make_plot(k1_rbf, k2_rbf, "rbf")
    make_plot(k1_mat12, k2_mat12, "matern")

    PlotSaver.finish(
        plots,
        relsize=0.33, ratio=1,
        ncols=3,
        save=False,
    )

with params.local():
    params.GRID_H = 0.3
    plot()

### Fig. 5.4: design-gp-warping-wraparound

In [ ]:
def plot():
    x = np.linspace(-2.5*np.pi, 2.5*np.pi, 500)
    f1 = np.abs(x)
    f2 = np.abs(2*np.sin(x/2))
    
    fig, axis = PlotSaver.create(
        xmargin=0,
        ylim=(0, 7),
        xticks=pi_ticks(0.5),
        yticks=[0,2,4,6],
    )
    axis.plot(x, f1, label=r"$\lvert\varphi - \varphi'\rvert$")
    axis.plot(x, f2, label=r"$2\left\lvert\sin\left(\frac{\varphi - \varphi'}{2}\right)\right\rvert$")
    axis.axvline(-2*np.pi, color="gray", linestyle="dashed")
    axis.axvline(2*np.pi, color="gray", linestyle="dashed")
    PlotSaver.finish(
        (fig, "design-gp-warping-wraparound"),
        relsize=0.6, legend=True,
        save=False,
    )
plot()

### Fig. 5.5: design-gp-periodic-summation

In [ ]:
def plot():
    r = np.linspace(-3*np.pi, 3*np.pi, 1000)[:, np.newaxis]
    k = gp.build_kernel_matern(nu=0.5)
    
    fig, axis = PlotSaver.create(
        xmargin=0,
        ylim=YLIM_1,
        xticks=pi_ticks(0.5),
        yticks=dict(ticks=[0, 1], labels=["0", r"$\sigma_f^2$"]),
    )
    axis.plot(r, gp.eval_kernel(k, r-2*np.pi), color="tab:blue")
    axis.plot(r, gp.eval_kernel(k, r        ), color="tab:blue")
    axis.plot(r, gp.eval_kernel(k, r+2*np.pi), color="tab:blue")
    axis.axvline(-2*np.pi, color="gray", linestyle="dashed")
    axis.axvline(2*np.pi, color="gray", linestyle="dashed")
    PlotSaver.finish(
        (fig, "design-gp-periodic-summation"),
        relsize=0.6,
        save=False,
    )
plot()

### Fig. 5.6: design-gp-truncation-function

In [ ]:
def plot():
    plots = []

    x = np.linspace(-2.5*np.pi, 2.5*np.pi, 500)
    c1, c2 = np.pi, 2*np.pi
    t = gp._build_truncation_function(c1, c2)
    k_normal = gp.build_kernel_matern(nu=0.5)
    k_trunc = gp.build_kernel_matern_periodic_truncated(nu=0.5, n=0)
    
    fig, axis = PlotSaver.create(
        xmargin=0,
        ylim=YLIM_1,
        xticks=pi_ticks(1),
        yticks=[0, 0.5, 1],
    )
    axis.plot(x, t(x), color="tab:orange", label=r"$t_{\pi\to 2\pi}(r)$")
    axis.fill_betweenx(YLIM_1, c1, c2, color="yellow", alpha=0.25, linestyles="dashed")
    axis.fill_betweenx(YLIM_1, -c2, -c1, color="yellow", alpha=0.25, linestyles="dashed")
    axis.axvline(-2*np.pi, color="gray", linestyle="dashed")
    axis.axvline(2*np.pi, color="gray", linestyle="dashed")
    plots.append((fig, "design-gp-truncation-function"))

    fig, axis = PlotSaver.create(
        xmargin=0,
        ylim=(YLIM_1[0], 0.025),
        xticks=pi_ticks(1),
    )
    axis.plot(x, gp.eval_kernel(k_normal, x[:,np.newaxis]), label=r"$k(r)$")
    axis.plot(x, gp.eval_kernel(k_trunc, x[:,np.newaxis]), label=r"$\tilde{k}(r)$")
    axis.fill_betweenx(YLIM_1, c1, c2, color="yellow", alpha=0.25, linestyles="dashed")
    axis.fill_betweenx(YLIM_1, -c2, -c1, color="yellow", alpha=0.25, linestyles="dashed")
    axis.axvline(-2*np.pi, color="gray", linestyle="dashed")
    axis.axvline(2*np.pi, color="gray", linestyle="dashed")
    plots.append((fig, "design-gp-truncated-kernel"))

    PlotSaver.finish(
        plots,
        relsize=0.45, legend=True,
        save=False,
    )
plot()

### Fig. 5.7: design-gp-periodic-matern

In [ ]:
def plot():
    r = np.linspace(-3.5*np.pi, 3.5*np.pi, 1000)[:, np.newaxis]
    k_warped = gp.build_kernel_matern_periodic_warped(nu=0.5)
    k_approx = gp.build_kernel_matern_periodic_approx(nu=0.5, n_approx=1)
    k_per12 = gp.build_kernel_matern_periodic(nu=0.5)
    
    fig, axis = PlotSaver.create(
        xmargin=0,
        ylim=YLIM_1,
        xticks=pi_ticks(0.5),
        yticks=dict(ticks=[0, 1], labels=["0", r"$\sigma_f^2$"]),
    )
    axis.plot(r, gp.eval_kernel(k_warped, r), label=r"$k_{M_{1/2}\text{-}p_u}$")
    axis.plot(r, gp.eval_kernel(k_per12, r), label=r"$k_{M_{1/2}\text{-}p_{\infty}}$")
    axis.plot(r, gp.eval_kernel(k_approx, r), label=r"$k_{M_{1/2}\text{-}\tilde{p}_{1}}$")
    axis.axvline(-2*np.pi, color="gray", linestyle="dashed")
    axis.axvline(2*np.pi, color="gray", linestyle="dashed")
    PlotSaver.finish(
        (fig, "design-gp-periodic-matern"),
        relsize=0.7, ratio=0.5, legend=dict(loc="upper right"),
        save=False,
    )
    
plot()

### Fig. 5.8: design-objective-ocl-ocu

In [ ]:
def plot():
    plots = []

    def make_plot(name, obj, thetas, xlim, ylim):
        cam = Camera()
        k = KERNEL()
        simulation = Simulation.build(obj, cam, k)
        simulation.run(thetas[:-1])
        simulation.move_camera(thetas[-1])

        plotter = SimulationPlotter(mode="real", undecorated=True)
        PlotSaver.set_style(
            plotter.axis,
            xlim=xlim,
            ylim=ylim,
            xlabel=r"$x$ coordinate",
            ylabel=r"$y$ coordinate",
        )
        plotter.plot_object(simulation.obj, show_object=False, show_bounds=False)
        plotter.plot_camera(simulation.camera)
        plotter.plot_confidence(simulation.algorithm.gp)
        plotter.plot_points(simulation.algorithm.observations.observed_points, color="green", markersize=2, name="previous")
        # true estimator
        estimates_opt = simulation.algorithm_opt.compute_estimate_points(simulation.camera)
        plotter.plot_points(estimates_opt, color="red", markersize=2, name="true")
        # lower confidence bound
        estimates = simulation.camera.compute_observation(simulation.algorithm.gp.lower_points)
        plotter.plot_points(estimates, color="dimgray", markersize=2, name="lower")
        # upper confidence bound
        estimates = simulation.camera.compute_observation(simulation.algorithm.gp.upper_points)
        plotter.plot_points(estimates, color="gray", markersize=2, name="upper")
        
        plots.append((plotter.fig, "design-objective-observation-{}".format(name)))

    make_plot(
        "ocl-ocu",
        obj=objects.PolygonObject.build("convex"),
        thetas=[1.76, 2.2],
        xlim=(-6.78, 2.01),
        ylim=(-1.45, 7.34),
    )
    make_plot(
        "counterexample",
        obj=objects.PolygonObject.build("convex"),
        thetas=[1.76, 2.04],
        xlim=(-6.78, 2.01),
        ylim=(-1.45, 7.34),
    )
    make_plot(
        "degenerate",
        obj=objects.PolygonObject.build("trident"),
        thetas=[0],
        xlim=(-0.58, 9.14),
        ylim=(-4.79, 4.94),
    )

    PlotSaver.finish(
        plots,
        relsize=0.33, ratio=1,
        save=False,
    )

with params.local():
    params.GRID_H = 0.4
    plot()

### Fig. 5.9: design-objective-fov-suminterval

In [ ]:
def plot():
    plots = []

    def make_plot_fov_boundary():
        obj = OBJECT()
        cam = CAMERA()
        k = KERNEL()
        simulation = Simulation.build(obj, cam, k)
        simulation.run(thetas[:-1])
        simulation.move_camera(thetas[-1])

        plotter = SimulationPlotter(mode="polar", undecorated=True)
        PlotSaver.set_style(
            plotter.axis,
            xticks=pi_ticks(0.5),
            ylabel=r"radial distance $r$",
        )
        plotter.plot_object(simulation.obj, show_object=False, show_bounds=False)
        plotter.plot_camera(simulation.camera)
        plotter.plot_confidence(simulation.algorithm.gp)
        plotter.plot_points(simulation.algorithm.observations.observed_points, color="green", markersize=2, name="previous")
        plotter.axis.annotate(r"$\theta$", (simulation.camera.theta-0.15, -1.2), annotation_clip=False)
        
        ray_style = dict(color="blue", linestyle="-", linewidth=1.5)
        ray_left = cam.camera_to_polar(params.CAM_FOV_RAD()/2, np.linspace(0, params.CAM_DOF, 100))
        ray_right = cam.camera_to_polar(-params.CAM_FOV_RAD()/2, np.linspace(0, params.CAM_DOF, 100))
        plotter.axis.plot(*ray_left, **ray_style)
        plotter.axis.plot(*ray_right, **ray_style)

        plots.append((plotter.fig, "design-objective-fov-boundary"))

    def make_plot_interval(interval, name):
        obj = OBJECT()
        cam = CAMERA()
        k = KERNEL()
        simulation = Simulation.build(obj, cam, k)
        simulation.run(thetas[:-1])
        simulation.move_camera(thetas[-1])

        plotter = SimulationPlotter(mode="polar", undecorated=True)
        PlotSaver.set_style(
            plotter.axis,
            xticks=pi_ticks(0.5),
            ylabel=r"radial distance $r$",
        )
        plotter.plot_object(simulation.obj, show_object=False, show_bounds=False)   
        plotter.plot_camera(simulation.camera)    
        plotter.plot_confidence(simulation.algorithm.gp, show_boundary=True)
        plotter.plot_points(simulation.algorithm.observations.observed_points, color="green", markersize=2, name="previous")
        plotter.axis.annotate(r"$\theta$", (simulation.camera.theta-0.15, -1.2), annotation_clip=False)
        
        c1, c2 = interval(simulation.camera, simulation.algorithm.gp)
        c1[0], c2[0] = c1[0] % (2*np.pi), c2[0] % (2*np.pi)
        plotter.plot_points(np.array([c1, c2]).T, color="blue", name="boundary_intersections")

        posA = (c1[0], np.min([c1[1], c2[1]]) - 1.7)
        posB = (c2[0], np.min([c1[1], c2[1]]) - 1.7)
        plotter.axis.add_patch(build_arrowpatch_length(posA, posB))

        plots.append((plotter.fig, "design-objective-suminterval-{}".format(name)))

    # define camera positions
    thetas = [3.10, 3.65]
    # plot FOV boundary
    make_plot_fov_boundary()
    # plot summation interval
    make_plot_interval(lambda camera, gp: objectives.Objective.get_simple_FOV_endpoint(camera), "intersect")
    make_plot_interval(objectives.Objective.get_FOV_confidence_intersection, "simple")

    PlotSaver.finish(
        plots,
        relsize=0.33, ratio=1,
        save=False,
    )

with params.local():
    params.GRID_H = 0.5
    plot()

### Fig. 5.10: design-objective-intersection / Fig. 5.11: design-objective-confidence / Fig. 5.12: design-objective-uncertainty

In [ ]:
def plot(alg_class):
    plots = []

    default_thetas = [3.8, 4.29]

    def make_plot(name, objective_class, algorithm_name, mode="real", thetas=None):
        if thetas is None:
            thetas = default_thetas
        
        # setup and run simulation
        cam = Camera()
        obj = OBJECT()
        k = KERNEL()
        simulation = Simulation.build(obj, cam, k, algorithm_name)
        simulation.run(thetas[:-1])
        simulation.move_camera(thetas[-1])
        
        # plot general components
        plotter = SimulationPlotter(mode=mode, undecorated=True)
        PlotSaver.set_style(
            plotter.axis,
            xticks=[] if mode == "real" else pi_ticks(2),
            xlabel=r"$x$ coordinate" if mode == "real" else "",
            ylabel=r"$y$ coordinate" if mode == "real" else r"radial distance $r$",
            xlim=(-7.88, 6.99) if mode == "real" else (2.63, 6.26),
            ylim=(-10.78, 4.08) if mode == "real" else (1.48, 8.47),
        )
        plotter.plot_object(simulation.obj, show_object=False, show_bounds=False)   
        plotter.plot_camera(simulation.camera)    
        plotter.plot_confidence(simulation.algorithm.gp, show_boundary=True)
        plotter.plot_points(simulation.algorithm.observations.observed_points, color="green", markersize=2, name="previous")
        # plot true objective
        estimates_opt = simulation.algorithm_opt.compute_estimate_points(simulation.camera)
        plotter.plot_points(estimates_opt, color="red", markersize=2, name="true")
        # plot objective function estimate
        estimates = simulation.algorithm.compute_estimate_points(simulation.camera)
        plotter.plot_points(estimates, color="red", markersize=1, alpha=0.25, name="estimate")
        # add annotations in polar world
        if mode == "polar":
            c1, c2 = simulation.algorithm.objective.get_summation_interval(simulation.camera, simulation.algorithm.data)
            c1[1] = np.min([c1[1], c2[1]]) - 0.4
            c2[1] = c1[1]
            plotter.axis.add_patch(build_arrowpatch_length(c1, c2))

            if objective_class == "uncertainty":
                lower, upper = simulation.algorithm.gp.confidence_boundary(simulation.camera.theta, interp=True)
                plotter.axis.add_patch(build_arrowpatch_length((simulation.camera.theta, lower), (simulation.camera.theta, upper)))
        
        plots.append((plotter.fig, "design-objective-{}-{}-{}".format(objective_class, name, mode)))

    if alg_class == "intersection":
        plots = []
        make_plot("ioa", "intersection", "Greedy-IntersectionOcclusionAware")
        make_plot("i", "intersection", "Greedy-Intersection")
        make_plot("ioa", "intersection", "Greedy-IntersectionOcclusionAware", mode="polar")
        make_plot("i", "intersection", "Greedy-Intersection", mode="polar")
        PlotSaver.finish(
            plots,
            relsize=0.33, ratio=1,
            ncols=2,
            save=False,
        )

    if alg_class == "confidence":
        plots = []
        make_plot("c", "confidence", "Greedy-Confidence")
        make_plot("cs", "confidence", "Greedy-ConfidenceSimple")
        plots[-1][0].gca().add_patch(build_arrowpatch_length(
            polar_to_cartesian(3/2*np.pi, params.CAM_D),
            polar_to_cartesian(3/2*np.pi, 0),
            linewidth=0.75,
        ))
        plots[-1][0].gca().add_patch(build_arrowpatch_length(
            polar_to_cartesian(3/2*np.pi, 4.5) + np.array([0.7, 0]),
            polar_to_cartesian(3/2*np.pi, 0) + np.array([0.7, 0]),
            linewidth=0.75,
        ))
        plots[-1][0].gca().annotate(r"$fov(\varphi;\theta)$", (1.5, -3.5))
        plots[-1][0].gca().annotate(r"$d_{cam}$", (0.7, -6.8))
        make_plot("csp", "confidence", "Greedy-ConfidenceSimplePolar")
        make_plot("c", "confidence", "Greedy-Confidence", mode="polar")
        make_plot("cs", "confidence", "Greedy-ConfidenceSimple", mode="polar")
        make_plot("csp", "confidence", "Greedy-ConfidenceSimplePolar", mode="polar")

        PlotSaver.finish(
            plots,
            relsize=0.33, ratio=1,
            ncols=3,
            save=False,
        )

    if alg_class == "uncertainty":
        plots = []
        make_plot("u", "uncertainty", "Greedy-Uncertainty")
        make_plot("u-max", "uncertainty", "Greedy-Uncertainty", thetas=[default_thetas[0], default_thetas[1]+0.5])
        make_plot("up", "uncertainty", "Greedy-UncertaintyPolar")
        make_plot("u", "uncertainty", "Greedy-Uncertainty", mode="polar")
        make_plot("u-max", "uncertainty", "Greedy-Uncertainty", mode="polar", thetas=[default_thetas[0], default_thetas[1]+0.5])
        make_plot("up", "uncertainty", "Greedy-UncertaintyPolar", mode="polar")
        PlotSaver.finish(
            plots,
            relsize=0.33, ratio=1,
            ncols=3,
            save=False,
        )

with params.local():
    params.GRID_H = 0.3
    plot("intersection")
    plot("confidence")
    plot("uncertainty")

### Fig. 5.13: design-algorithm-confidence-vs-uncertainty

In [ ]:
def plot():
    plots = []

    thetas = [0.5, 1.05, 1.24, 2.92, 3.08, 4.33] + [0]
    simulation = Simulation.build(OBJECT(), Camera(), KERNEL(), "Greedy-Uncertainty")
    simulation.run(thetas[:-1])
    simulation.move_camera(thetas[-1])
    
    # artificially reduce uncertainty
    points_mask = is_in_range(simulation.obj.surface_points[0], (np.pi/2, np.pi), mod=2*np.pi)
    simulation.algorithm.gp.update(*simulation.obj.surface_points[:, points_mask], noise=4)

    plotter = SimulationPlotter(mode="polar", undecorated=True)
    PlotSaver.set_style(
        plotter.axis,
        xticks=pi_ticks(2),
        xlabel="",
        ylabel=r"radial distance $r$",
        ylim=(1, 10)
    )
    # plot general components
    plotter.plot_confidence(simulation.algorithm.gp)
    plotter.plot_points(simulation.algorithm.observations.observed_points, color="green", markersize=2, name="previous")
    # plot annotations
    nbv = simulation.algorithm.compute_nbv()
    plotter.axis.add_patch(build_arrowpatch_length((nbv, 2.5), (nbv, 8.2), linewidth=0.75))
    plotter.axis.annotate(r"(a)", (0.7*np.pi, 7), xytext=(3/4*np.pi, 8.5), arrowprops=arrowprops(rad=0.2))
    plotter.axis.annotate(r"(b)", (nbv+0.05, 6.6), xytext=(5/4*np.pi, 7.5), arrowprops=arrowprops(rad=-0.2))

    plots.append((plotter.fig, "design-algorithm-confidence-vs-uncertainty"))

    PlotSaver.finish(
        plots,
        relsize=0.7,
        save=False,
    )

with params.local():
    params.GRID_H = 0.5
    plot()

## Plots for Chapter 7

### Fig. 7.1: experiments-framework-objects

In [ ]:
def plot():
    plots = []

    def make_plot(obj, name):
        for mode in ["real", "polar"]:
            plotter = SimulationPlotter(mode=mode, undecorated=True)
            PlotSaver.set_style(
                plotter.axis,
                xticks=[] if mode == "real" else pi_ticks(),
                xlabel=r"$x$ coordinate" if mode == "real" else "",
                ylabel=r"$y$ coordinate" if mode == "real" else r"radial distance $r$",
            )
            plotter.plot_object(obj)
            plots.append((plotter.fig, "experiments-framework-objects-{}-{}".format(name, mode)))
    
    make_plot(objects.EllipseObject(a=6, b=4), "ellipse")
    plots[-2][0].gca().add_patch(build_arrowpatch_length((0, 0), (6, 0), linewidth=0.75))
    plots[-2][0].gca().add_patch(build_arrowpatch_length((0, 0), (0, 4), linewidth=0.75))
    plots[-2][0].gca().annotate(r"$a$", (2.5, -2))
    plots[-2][0].gca().annotate(r"$b$", (-2, 1))

    make_plot(objects.FlowerObject(frequency=5, amplitude=1.5), "flower")
    plots[-1][0].gca().add_patch(build_arrowpatch_length((np.pi*6/5, params.OBJ_D_AVG()), (np.pi*6/5, params.OBJ_D_AVG()+1.5), linewidth=0.75))
    plots[-1][0].gca().annotate(r"$a$", (np.pi-0.1, params.OBJ_D_AVG() + 0.5))
    
    make_plot(objects.SquareObject(width=8), "square")
    plots[-2][0].gca().add_patch(build_arrowpatch_length((-4, -5.5), (4, -5.5), linewidth=0.75))
    plots[-2][0].gca().annotate(r"$w$", (-0.7, -7.5))
    
    make_plot(objects.PolygonObject.build("convex"), "polygon")

    PlotSaver.finish(
        plots[0::2] + plots[1::2],
        relsize=0.25, ratio=1,
        save=False,
    )
plot()

### Fig. 7.2: experiments-results-deficiency

In [ ]:
def plot():
    plots = []

    def make_plot(name):
        plotter = SimulationPlotter(mode="polar", undecorated=True)
        PlotSaver.set_style(
            plotter.axis,
            xticks=pi_ticks(2),
            ylabel=r"radial distance $r$",
            xlim=(0.78, 5.07),
            ylim=(1.36, 9.60),
        )
        # plot general components
        plotter.plot_object(simulation.obj, show_object=False, show_bounds=False)   
        plotter.plot_camera(simulation.camera)    
        plotter.plot_confidence(simulation.algorithm.gp, show_boundary=True)
        plotter.plot_points(simulation.algorithm.observations.observed_points, color="green", markersize=2, name="previous")
        # plot objective function estimate
        estimates = simulation.algorithm.compute_estimate_points(simulation.camera)
        plotter.plot_points(estimates, color="red", markersize=1, alpha=0.25, name="estimate")
        
        plots.append((plotter.fig, "experiments-results-deficiency-{}".format(name)))
    
    # setup simulation
    cam = Camera()
    obj = OBJECT()
    k = KERNEL()
    simulation = Simulation.build(obj, cam, k, "Greedy-ConfidenceSimple")
    # run simulation
    thetas = [1, 5, 2.6, 2.6]
    simulation.run(thetas[:-1])
    simulation.move_camera(thetas[-1])

    # plot at visited location
    make_plot("a")
    # plot at new location
    simulation.move_camera(simulation.camera.theta + 1)
    make_plot("b")
    # plot estimates
    x, y = simulation.algorithm.thetas, simulation.algorithm.compute_nbv(with_estimates=True)[1]
    plotter = SimulationPlotter(mode="polar", undecorated=True)
    PlotSaver.set_style(
        plotter.axis,
        xticks=pi_ticks(2),
        ylabel=r"objective $F_u(\theta\mid\theta_{1:t-1})$",
        xlim=(0.78, 5.07),
        ylim=(3, 10.5),
    )
    plotter.axis.plot(x, y/np.max(y) * 0.9 * params.CAM_D)
    plotter.plot_camera(simulation.camera, show_fov=False, show_view_circle=False, show_los="position")
    plots.append((plotter.fig, "experiments-results-deficiency-c"))

    PlotSaver.finish(
        plots,
        relsize=0.33, ratio=1,
        save=False,
    )
plot()

## Plots for Chapter B

### Fig. B.1: simulation-experiment-objects

In [ ]:
def plot():

    def make_plot(obj):
        plotter = SimulationPlotter(mode="real", undecorated=True)
        PlotSaver.set_style(
            plotter.axis,
            xlim=(-params.OBJ_D_MAX-2, params.OBJ_D_MAX+2),
            ylim=(-params.OBJ_D_MAX-2, params.OBJ_D_MAX+2),
        )
        plotter.plot_object(obj)
        plots.append((plotter.fig, "simulation-experiment-objects-{}".format(str(obj))))
    
    plots = []
    make_plot(objects.EllipseObject(a=5, b=5))
    make_plot(objects.EllipseObject(a=6, b=4))
    make_plot(objects.EllipseObject(a=7, b=3))
    make_plot(objects.EllipseObject(a=8, b=2))
    PlotSaver.finish(
        plots,
        relsize=0.2, ratio=1,
        save=False,
    )

    plots = []
    make_plot(objects.FlowerObject(frequency=1, amplitude=2))
    make_plot(objects.FlowerObject(frequency=2, amplitude=3))
    make_plot(objects.FlowerObject(frequency=3, amplitude=1))
    make_plot(objects.FlowerObject(frequency=3, amplitude=2))
    make_plot(objects.FlowerObject(frequency=4, amplitude=1))
    make_plot(objects.FlowerObject(frequency=4, amplitude=2))
    make_plot(objects.FlowerObject(frequency=5, amplitude=1))
    make_plot(objects.FlowerObject(frequency=5, amplitude=2))
    make_plot(objects.FlowerObject(frequency=8, amplitude=1))
    make_plot(objects.FlowerObject(frequency=8, amplitude=2))
    PlotSaver.finish(
        plots,
        relsize=0.2, ratio=1,
        save=False,
    )
    
    plots = []
    make_plot(objects.SquareObject(width=4))
    make_plot(objects.SquareObject(width=6))
    make_plot(objects.SquareObject(width=8))
    make_plot(objects.SquareObject(width=10))
    PlotSaver.finish(
        plots,
        relsize=0.2, ratio=1,
        save=False,
    )
    
    plots = []
    make_plot(objects.PolygonObject.build("diamond"))
    make_plot(objects.PolygonObject.build("convex"))
    make_plot(objects.PolygonObject.build("concave"))
    make_plot(objects.PolygonObject.build("star"))
    PlotSaver.finish(
        plots,
        relsize=0.2, ratio=1,
        save=False,
    )

plot()

## Other Plots

In [ ]:
PlotSaver.disable_save_all()

### Regret Bounds

In [ ]:
from algorithms.analysis import Analysis
def plot():
    t = np.arange(0, 50)+1
    fig, ax = plt.subplots(ncols=2, nrows=3, figsize=(8, 8), constrained_layout=True)
    ax[0, 0].plot(t, Analysis(nu=2.5, l=1, delta=0.5).beta_bound(t), label=r"$\delta=0.5$")
    ax[0, 0].plot(t, Analysis(nu=2.5, l=1, delta=0.25).beta_bound(t), label=r"$\delta=0.25$")
    ax[0, 0].plot(t, Analysis(nu=2.5, l=1, delta=0.1).beta_bound(t), label=r"$\delta=0.1$")
    PlotSaver.set_style(ax[0, 0], title="Beta Bound", legend="True")
    ax[0, 1].plot(t, Analysis(nu=2.5, l=1).gamma_bound(t), label=r"$\nu=5/2$")
    ax[0, 1].plot(t, Analysis(nu=1.5, l=1).gamma_bound(t), label=r"$\nu=3/2$")
    ax[0, 1].plot(t, Analysis(nu=0.5, l=1).gamma_bound(t), label=r"$\nu=1/2$")
    PlotSaver.set_style(ax[0, 1], title="Gamma Bound", legend="True")

    ax[1, 0].plot(t, Analysis(nu=2.5, l=1).avg_regret_bound(t, with_constants=False), label=r"$\nu=5/2$")
    ax[1, 0].plot(t, Analysis(nu=1.5, l=1).avg_regret_bound(t, with_constants=False), label=r"$\nu=3/2$")
    ax[1, 0].plot(t, Analysis(nu=0.5, l=1).avg_regret_bound(t, with_constants=False), label=r"$\nu=1/2$")
    PlotSaver.set_style(ax[1, 0], title="Average Regret Bound (without constants)", legend=True)
    ax[1, 1].plot(t, Analysis(nu=2.5, l=1).avg_regret_bound(t, with_constants=False), label=r"$l=1$")
    ax[1, 1].plot(t, Analysis(nu=2.5, l=0.5).avg_regret_bound(t, with_constants=False), label=r"$l=0.5$")
    ax[1, 1].plot(t, Analysis(nu=2.5, l=0.2).avg_regret_bound(t, with_constants=False), label=r"$l=0.2$")
    PlotSaver.set_style(ax[1, 1], title="Average Regret Bound (without constants)", legend=True)
    
    ax[2, 0].plot(t, Analysis(nu=2.5, l=1, sigma_f=1).avg_regret_bound(t, with_constants=False), label=r"$\sigma_f = 1$")
    ax[2, 0].plot(t, Analysis(nu=2.5, l=1, sigma_f=1.5).avg_regret_bound(t, with_constants=False), label=r"$\sigma_f = 1.5$")
    ax[2, 0].plot(t, Analysis(nu=2.5, l=1, sigma_f=2).avg_regret_bound(t, with_constants=False), label=r"$\sigma_f = 2$")
    PlotSaver.set_style(ax[2, 0], title="Average Regret Bound (without constants)", legend=True)
    ax[2, 1].plot(t, Analysis(nu=2.5, l=1, sigma_eps=0.2).avg_regret_bound(t, with_constants=False), label=r"$\sigma_\varepsilon=0.2$")
    ax[2, 1].plot(t, Analysis(nu=2.5, l=1, sigma_eps=0.4).avg_regret_bound(t, with_constants=False), label=r"$\sigma_\varepsilon=0.4$")
    ax[2, 1].plot(t, Analysis(nu=2.5, l=1, sigma_eps=0.6).avg_regret_bound(t, with_constants=False), label=r"$\sigma_\varepsilon=0.6$")
    PlotSaver.set_style(ax[2, 1], title="Average Regret Bound (without constants)", legend=True)
    display(fig)
plot()

### Simulation GIF

In [ ]:
from tqdm import tqdm

def plot():
    gif_saver = GIFSaver(PlotSaver.output, "simulation")
    
    def pi_ticks(denominator=1):
        ticks = MultipleTicks(denominator=denominator, number=np.pi, latex="\pi", number_in_frac=False)
        return dict(locator=ticks.locator(), formatter=ticks.formatter())

    def make_world_plot(axis, mode):
        plotter = SimulationPlotter(mode=mode)
        plt.close(plotter.fig)
        plotter.axis = axis # overwrite internal axis
        rlim = params.CAM_D * 1.1
        if mode == "real":
            PlotSaver.set_style(
                plotter.axis,
                title="Real World",
                xlabel="x coordinates [m]",
                ylabel="y coordinates [m]",
                xlim=(-rlim, rlim),
                ylim=(-rlim, rlim),
                xticks=[-10, -5, 0, 5, 10],
                yticks=[-10, -5, 0, 5, 10],
            )
            axis.set_aspect("equal")
        elif mode == "polar":
            PlotSaver.set_style(
                plotter.axis,
                title="Polar World",
                xlabel="polar angle [rad]",
                ylabel="radial distance [m]",
                xlim=[0, 2*np.pi],
                ylim=[0, rlim],
                xticks=pi_ticks(2),
                yticks=[0, 5, 10],
            )
        plotter.plot_camera(simulation.camera)    
        plotter.plot_confidence(simulation.algorithm.gp)
        plotter.plot_points(simulation.algorithm.observations.observed_points, color="green", markersize=1, name="previous")
        plotter.plot_points(simulation.algorithm.compute_estimate_points(simulation.camera), color="red", markersize=1, name="current")

    ymax = None
    def make_estimates_plot(axis):
        nonlocal ymax
        x, y = simulation.algorithm.thetas, simulation.algorithm.compute_nbv(with_estimates=True)[1]
        if ymax is None:
            ymax = np.max(y)
        
        PlotSaver.set_style(
            axis,
            xticks=pi_ticks(2),
            title="Objective Function",
            xlabel="polar angle [rad]",
            xlim=[0, 2*np.pi],
            ylim=[0, ymax*1.2]
        )
        # plot estimates
        axis.plot(x, y, color="red")
        # plot camera
        plotter = SimulationPlotter(mode="polar")
        plt.close(plotter.fig)
        plotter.axis = axis.twinx() # overwrite internal axis
        plotter.axis.set_ylim([0, params.CAM_D*1.1])
        plotter.plot_camera(simulation.camera, show_fov=False, show_view_circle=False, show_los="position")
    
    def make_progress_plot(axis):
        results = simulation.results()
        x, y = results.rounds, results.n_total_rel
        
        PlotSaver.set_style(
            axis,
            title="Reconstruction Progress",
            xlabel="rounds",
            ylabel="progress",
            xlim=[0, np.max(x, initial=10) + 2],
            ylim=[0, 1.2],
        )
        axis.axhline(1, color="limegreen", linestyle="--", linewidth=1)
        axis.plot(x, y, marker="o", markersize=4, color="limegreen")
        x_finish = x[y == 1][:1]
        axis.plot(x_finish, np.full_like(x_finish, 1), marker="*", markersize=10, color="limegreen")

    def make_frame(t):
        fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(6, 4.5), constrained_layout=True, gridspec_kw={"height_ratios": [2, 1]})
        fig.suptitle("Simulation $t$={}".format(t), fontsize=10)
        make_world_plot(axes[0, 0], "real")
        make_world_plot(axes[0, 1], "polar")
        make_progress_plot(axes[1, 0])
        make_estimates_plot(axes[1, 1])
        gif_saver.add_frame(fig)
        plt.close(fig)

    # setup simulation
    cam = Camera()
    obj = objects.PolygonObject.build("convex")
    k = gp.build_kernel_matern_periodic(sigma=1.5, l=0.2, nu=1.5)
    simulation = Simulation.build(obj, cam, k, TRUE_ALGORITHM)

    # run simulation and make frames
    with tqdm(total=100, unit="%", desc="Simulation", bar_format="{desc} |{bar}| {n:.1f}{unit} [{elapsed}<{remaining}{postfix}]") as tqdm_steps:
        t = 1
        while not simulation.is_converged():
            simulation.move_camera(simulation.algorithm.compute_nbv())
            make_frame(t)
            simulation.take_measurement()
            make_frame(t)
            t += 1
            tqdm_steps.update(simulation.progress() * 100 - tqdm_steps.n)
    
    # save GIF
    gif_saver.finish(duration=1000, loop=0)

with PlotSaver.use_default_style():
    plt.rcParams.update({
        "font.size": 8,
        "axes.titlesize": 8,
        "axes.labelsize": 8,
        "xtick.labelsize": 8,
        "ytick.labelsize": 8,
        "legend.fontsize": 8,
    })
    plot()